<a href="https://colab.research.google.com/github/molley999/Review_Data_Generator/blob/main/Review_Data_Generator3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0
!pip install transformers soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import pipeline
import soundfile as sf
import numpy as np
from transformers import pipeline
import soundfile as sf, os, tempfile
from IPython.display import Audio

from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, BitsAndBytesConfig
import torch
import gc
from IPython.display import Markdown, display, update_display
import gradio as gr
import random
import tempfile, os

from threading import Thread


In [4]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

MODELS = {
    'enthusiastic': 'microsoft/Phi-3-mini-4k-instruct',
    'balanced': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
    'casual': 'google/gemma-2-2b-it'
}


quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

LOADED_MODELS = {}
for model_name, model_path in MODELS.items():
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map = "auto",
        quantization_config=quant_config
        )

    LOADED_MODELS[model_name] = {
        'model': model,
        'tokenizer': tokenizer
    }



def load_model(model_name):
    """Retrieve pre-loaded model from cache"""
    return LOADED_MODELS[model_name]['model'], LOADED_MODELS[model_name]['tokenizer']


def create_messages(model_name, system_msg, user_msg):
    # Gemma models don't support "system" role
    if "gemma" in model_name.lower():
        return [{"role": "user", "content": system_msg + " " + user_msg}]
    else:
        return [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ]


def create_review_prompt(product_name, category, star_rating, review_style):
  """Create different prompts based on review style"""
  if review_style == 'enthusiatic':
    system_msg = "You are an excited customer who loves sharing detailed positive experiences."
    user_msg = f"Write an {review_style} {star_rating}-star review for {product_name} ({category}). Be specific about features you loved. Keep under 100 words."

  elif review_style == 'balanced':
    system_msg = "You are a thoughtful reviewer who gives honest, balanced feedback."
    user_msg = f"Write a {star_rating}-star review for {product_name} ({category}). Include pros and cons. Be helpful and honest. Keep under 100 words."

  else:
    system_msg = "You are a casual reviewer who writes short, informal reviews."
    user_msg = f"Write a brief {star_rating}-star review for {product_name} ({category}). Use casual language like texting a friend. Keep under 50 words."

  return system_msg, user_msg


def stream_review(product_name, category, star_rating, review_style):
    """Generate one review using specified model"""

    model, tokenizer = load_model(review_style)

    system_msg, user_msg = create_review_prompt(product_name, category, star_rating, review_style)

    # messages = [
    # {"role": "system", "content": system_msg},
    # {"role": "user", "content": user_msg}
    #  ]

    messages = create_messages(MODELS[review_style], system_msg, user_msg)

    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        padding=True,
        add_generation_prompt=True,
        truncation=True
        ).to("cuda")


    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = {
        "input_ids": inputs,
        "max_new_tokens": 150,
        "temperature": 0.7,
        "do_sample": True,
        "streamer": streamer
    }

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()


    # Yield tokens as they're generated
    for new_text in streamer:
        yield new_text

    thread.join()

    # del model, tokenizer, inputs, generated_ids
    # torch.cuda.empty_cache()



tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [5]:


# load once (outside gradio_stream to save time)
tts_pipe = pipeline("text-to-speech", model="facebook/mms-tts-eng", device="cuda")

def text_to_speech(text: str):
    speech = tts_pipe(text)

    # Ensure audio is 1D
    audio = np.array(speech["audio"]).squeeze()

    # Save as 16-bit PCM WAV
    tmp_wav = os.path.join(tempfile.gettempdir(), "review.wav")
    sf.write(tmp_wav, audio, speech["sampling_rate"], subtype="PCM_16")
    return tmp_wav


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

Device set to use cuda


In [6]:
def gradio_stream(product_name, category, num_reviews, include_ratings):
    """Main function called by Gradio interface"""

    if not product_name.strip():
      yield "Pleasee enter a product name!", None
      return

    styles = ['enthusiastic', 'balanced', 'casual']
    all_reviews = ""
    final_audio = None

    for i in range(int(num_reviews)):
      if include_ratings:
        star_rating = random.choices([5, 4, 3, 2, 1], weights=[40, 30, 15, 10, 5])[0]
      else:
        star_rating = random.choices([5, 4, 3], weights=[50, 30, 20])[0]

      style = styles[i % len(styles)]
      stars = "⭐" * star_rating

      try:
        review_text = ""
        for word in stream_review(product_name, category, star_rating, style):
          review_text += word
          current_entry = f"{stars} ({star_rating}/5)\n{review_text}\n"
          if all_reviews:
            yield all_reviews + "\n" + "="*50 + "\n" + current_entry, None
          else:
            yield current_entry, None

        if all_reviews:
          all_reviews += "\n" + "="*50 + "\n" + current_entry
        else:
          all_reviews = current_entry

      except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        all_reviews += f"\n{error_msg}\n"
        yield all_reviews, None


    if all_reviews.strip():
        tmp_wav = text_to_speech(all_reviews)
        yield all_reviews, tmp_wav
    else:
        yield "❌ No review generated", None



In [8]:

# Sample products for testing
sample_products = [
    "WiFi-enabled Smart Toaster",
    "Self-Stirring Coffee Mug",
    "Bluetooth Shower Curtain",
    "AI-Powered Houseplant",
    "Voice-Activated Umbrella"
]


with gr.Blocks(css="""
    .gr-box {border-radius: 10px;}
    .gr-button {background: linear-gradient(90deg, #4F46E5, #6366F1); color: white;}
    .gr-button:hover {opacity: 0.9;}
""") as demo:

    gr.Markdown(
        "<h1 style='text-align:center;'>🌟 AI Review Generator</h1>"
        "<p style='text-align:center;'>Generate realistic product reviews using multiple AI models.<br>"
        "Perfect for testing e-commerce sites!</p>"
    )

    with gr.Row():
        # Left column (inputs)
        with gr.Column(scale=1):
            # gr.Markdown("### ✍️ Input")
            product = gr.Textbox(
                label="Product Name",
                placeholder="Enter any product name...",
                value="Smart Bluetooth Toothbrush"
            )

            category = gr.Dropdown(
                choices=["Electronics", "Home & Garden", "Books", "Clothing", "Food & Drink", "Sports", "Toys", "Other"],
                label="Product Category",
                value="Electronics"
            )

            num_reviews = gr.Slider(
                minimum=1, maximum=5, value=3, step=1,
                label="Number of Reviews"
            )

            include_negative = gr.Checkbox(
                label="Include negative reviews (1-2 stars)", value=False
            )

            generate_btn = gr.Button("Generate Reviews")

        # Right column (outputs)
        with gr.Column(scale=1):
            # gr.Markdown("### 📤 Output")
            out_text = gr.Textbox(
                label="Generated Reviews",
                lines=15,
                placeholder="Generated reviews will appear here..."
            )
            out_audio = gr.Audio(label="Audio Review")

    # Examples across both columns
    # gr.Markdown("### 💡 Examples")
    gr.Examples(
        examples=[
            ["WiFi-enabled Smart Toaster", "Electronics", 3, False],
            ["Self-Stirring Coffee Mug", "Home & Garden", 4, True],
            ["AI-Powered Houseplant", "Home & Garden", 2, False]
        ],
        inputs=[product, category, num_reviews, include_negative]
    )

    # Button action
    generate_btn.click(
        fn=gradio_stream,
        inputs=[product, category, num_reviews, include_negative],
        outputs=[out_text, out_audio]
    )


# Launch
if __name__ == "__main__":
    demo.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://727e3b634125f93622.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://727e3b634125f93622.gradio.live
